# Lesson 6: Essay Writer

In [ ]:
from dotenv import load_dotenv

_ = load_dotenv()

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

In [ ]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int
    error_code: int
    sop_content: str
    available_tools: List[str]

In [ ]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
def _extract_json_from_response(self, response_content: str) -> Dict[str, Any]:
        """Extracts JSON from the LLM response."""
        json_pattern = r'```json\s*(.*?)\s*```'
        json_match = re.search(json_pattern, response_content, re.DOTALL)
        
        if json_match:
            json_str = json_match.group(1)
        else:
            json_str = response_content.strip()
        
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            json_start = response_content.find('{')
            json_end = response_content.rfind('}') + 1
            
            if json_start != -1 and json_end > json_start:
                json_str = response_content[json_start:json_end]
                return json.loads(json_str)
            else:
                raise ValueError(f"Could not extract valid JSON from response: {str(e)}")


In [ ]:
def planner(llm,state: AgentState) -> AgentState:
    error_code = state["error_code"]
    sop_content = state["sop_content"]
    available_tools = state["available_tools"]
    tool_descriptions = []
    for tool in available_tools:
        tool_descriptions.append(f"- {tool.name}: {tool.description}")
    
    planner_prompt = f"""
        You are an expert SOP analyzer that creates ReAct-optimized playbooks for intelligent execution agents.
        
        Your task is to analyze the SOP and create a JSON playbook where each step provides enough context 
        for a ReAct agent to intelligently choose tools and make decisions based on tool outcomes.
        
        AVAILABLE TOOLS:
        {chr(10).join(tool_descriptions)}
        
        CRITICAL DESIGN PRINCIPLES:
        1. Each step should define OBJECTIVES, not specific tools to use
        2. Include clear SUCCESS and FAILURE criteria that a ReAct agent can evaluate
        3. Steps should be tool-agnostic - let the ReAct agent choose the best tool for the objective
        4. Provide rich context about what each step is trying to achieve
        5. Design conditional flows that can handle multiple scenarios
        
        Guidelines for ReAct-optimized playbooks:
        1. Focus on WHAT needs to be achieved, not HOW to achieve it
        2. Provide clear success/failure criteria for each step
        3. Include context requirements (what data/information is needed)
        4. Design steps that allow the ReAct agent to reason about tool selection
        5. Create robust error handling and alternative paths
        6. Each step should be self-contained with clear objectives
        
        Error Code Context: {error_code}
        
        IMPORTANT: Return ONLY the JSON playbook, no other text or explanations.
        """
        
    human_prompt = f"""
        Create a ReAct-optimized JSON playbook for this SOP:
        
        {sop_content}
        
        Required JSON structure:
        {{
            "name": "Playbook name",
            "description": "What this playbook accomplishes",
            "start_step": "first_step_id",
            "steps": {{
                "step_id": {{
                    "id": "step_id",
                    "name": "Step name",
                    "action": "achieve_objective|evaluate_condition|notify|end",
                    "objective": "What this step needs to accomplish",
                    "success_criteria": "How to determine success",
                    "failure_criteria": "How to determine failure", 
                    "next_steps": {{"success": "next_id", "failure": "error_id", "default": "END"}},
                    "description": "Detailed description",
                    "context_requirements": ["required_data1", "required_data2"]
                }}
            }}
        }}
        
        RULES:
        1. Use "END" for terminal steps
        2. Focus on objectives, not specific tools
        3. Include rich success/failure criteria
        4. All referenced step IDs must exist
        5. Make it ReAct agent friendly - provide reasoning context
        """
        
    messages = [
            SystemMessage(content=planner_prompt),
            HumanMessage(content=human_prompt)
        ]
    response = llm.invoke(messages)
    playbook_json = _extract_json_from_response(response.content)
    state["plan"] = eval(playbook_json)
    state["sop_content"] = None

    return state

In [ ]:
def call_llm(llm,tools,state: AgentState) -> AgentState:
    """Function to call the LLM with the current state."""
    llm = llm.bind_tools(tools)

    react_template = f"""You are an intelligent workflow execution agent. You have access to the following tools:

{tools}

Your role is to execute workflow steps by:
1. Understanding the objective of each step
2. Choosing the most appropriate tool(s) to achieve the objective
3. Evaluating tool results against success/failure criteria
4. Making decisions about next steps based on outcomes

Use the following format:

Question: the workflow step you need to execute
Thought: analyze the objective and determine what needs to be done
Action: choose the most appropriate action from [{tools}]
Action Input: provide the necessary input for the action
Observation: analyze the result of the action
... (repeat Thought/Action/Action Input/Observation as needed)
Thought: evaluate if the step objective has been achieved based on success/failure criteria
Final Answer: provide a clear assessment of whether the step succeeded or failed, with reasoning

CRITICAL RULES:
1. If no suitable tool exists for an objective, respond with "NO_SUITABLE_TOOL"
2. If a tool fails or returns unsatisfactory results, try alternative approaches if available
3. Always evaluate results against the provided success/failure criteria
4. If you cannot achieve the objective after trying available tools, respond with "OBJECTIVE_FAILED"

Begin! """
    response = llm.invoke().content
    return state

In [ ]:
def take_action(state: AgentState) -> AgentState:
    """Execute tool calls from the LLM's response."""

    tool_calls = state['messages'][-1].tool_calls
    results = []
    for t in tool_calls:
        print(f"Calling Tool: {t['name']} with query: {t['args'].get('query', 'No query provided')}")
        
        if not t['name'] in tools_dict: # Checks if a valid tool is present
            print(f"\nTool: {t['name']} does not exist.")
            result = "Incorrect Tool Name, Please Retry and Select tool from List of Available tools."
        
        else:
            result = tools_dict[t['name']].invoke(t['args'].get('query', ''))
            print(f"Result length: {len(str(result))}")
            

        # Appends the Tool Message
        results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))

    print("Tools Execution Complete. Back to the model!")
    return {'messages': results}

In [ ]:
graph = StateGraph(AgentState)
graph.add_node("planner", planner)
graph.add_node("llm", call_llm)
graph.add_node("tools", take_action)

graph.add_conditional_edges(
    "llm",
    should_continue,
    {True: "tools", False: END}
)
graph.add_edge("planner", "llm")
graph.add_edge("retriever_agent", "llm")
graph.set_entry_point("planner")



In [ ]:
agent = graph.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image

Image(agent.get_graph().draw_png())

In [ ]:
thread = {"configurable": {"thread_id": "1"}}
for s in agent.stream({
    'task': "what is the difference between langchain and langsmith",
    "max_revisions": 2,
    "revision_number": 1,
}, thread):
    print(s)

## Essay Writer Interface

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from helper import ewriter, writer_gui

In [ ]:
MultiAgent = ewriter()
app = writer_gui(MultiAgent.agent)
app.launch()